# METRICAS PAPER

In [ ]:
import re

## APIPE (Average Precision in Exact Passage Extraction)

In [ ]:
retrieved_chunks=0
ground_truth_field=0 ## Información que buscamos como ID o el precio

hits = [1 if ground_truth_field in chunk else 0 for chunk in retrieved_chunks]
precision = sum(hits) / len(hits) if hits else 0


## APKAC (Average Precision in K-chunk Answer Context)

In [ ]:
retrieved_chunks=0
supporting_context=0 # fragmento donde se encuentra la respuesta

hits = [1 if supporting_context in chunk else 0 for chunk in retrieved_chunks]
precision = sum(hits) / len(hits) if hits else 0

# Evaluación Automática con LLM

In [ ]:
def extract_score_from_response(response_text):
    match = re.search(r"[Pp]untuaci[oó]n\s*[:=]?\s*(\d)", response_text)
    if match:
        return int(match.group(1))
    else:
        return None 

## Factual Consistency Score (FCS)

In [ ]:
import langchain
num_queries = 25
result={}
scores={}    
for i in range(1, num_queries + 1):
    question = f"query_{i}"
    answer=f"answer_{i}"
    context=f"retrieved_chunks_{i}"
    
    prompt = f"""
    Eres un evaluador experto. Vas a evaluar si una respuesta está completamente justificada por un contexto.

    Pregunta: {question}

    Contexto recuperado:
    {context}

    Respuesta generada por el sistema:
    {answer}

    ¿Está la respuesta completamente basada en el contexto? Evalúa del 1 al 5, donde:
    1 = inventada completamente,
    5 = totalmente basada en el contexto.

    Explica brevemente tu decisión y da la puntuación al final.

    Formato de salida:
    Justificación: ...
    Puntuación: <número del 1 al 5>
    """
    
    results[i] = llm.invoke(prompt)  # o llm(prompt)
    scores[i] = extract_score_from_response(results[i])

    print(f"Puntuación consulta {i}: {scores[i]}")
    print(f"Explicación de LLaMA:\n {results[i]}")
    print("\n\n---\n\n")
    
    rows.append({
        "query_id": i,
        "question": question,
        "answer": answer,
        "retrieved_chunks": context,
        "llm_response": response,
        "score": score
    })

Crear el CSV

In [ ]:
df = pd.DataFrame(rows)
df.to_csv("evaluacion_rag_scores.csv", index=False)

Visualización

In [ ]:
# Visualizaciones
plt.figure(figsize=(12, 5))

# Histograma
plt.subplot(1, 2, 1)
sns.histplot(df["score"], bins=5, kde=True)
plt.title("Distribución de Puntuaciones (Histograma)")
plt.xlabel("Puntuación")
plt.ylabel("Frecuencia")

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(x=df["score"])
plt.title("Boxplot de Puntuaciones")
plt.xlabel("Puntuación")

plt.tight_layout()
plt.show()

## Sub-question Coverage

Voy a utilizar las consultas generadas a través de multiquery y a partir de estas realizar está metrica

Este codigo puedo tambien añadirlo antes de las subconsultas del multiquery haciendo una evaluación como la de antes

In [ ]:
def extract_score_from_response(response_text):
    match = re.search(r"[Pp]untuaci[oó]n\s*[:=]?\s*(\d)", response_text)
    if match:
        return int(match.group(1))
    else:
        return None 
    
num_queries = 25
result={}
scores={}    
for i in range(1, num_queries + 1):
    
    question = f"query_{i}"
    retrieved_docs = retriever_multi_7_txt.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs]) # Se puede limitar el contexto
    answer = rag_chain_multi_7_txt.invoke(query)
    
    sub_questions = retriever_multi_7_txt.generate_queries(query, 3)
    sub_evals = {}
    for i, sub_q in enumerate(sub_questions):
        prompt = f"""Evalúa del 1 al 5 si esta respuesta responde a la pregunta usando SOLO el contexto:

        Pregunta: {question}
        Contexto: {context}
        Respuesta: {answer}

        Devuelve SOLO el número (1=inventada, 5=exacta):"""
    
        eval_result[i] = llm.invoke(prompt)
        sub_evals[i] = extract_score_from_response(eval_result[i])      
        print(f"La consulta {i} tiene las subconsulta {sub_q} con una nota de {sub_evals}")
        
    print("\n---\n")